# DNBLab Jupyter Notebook Tutorial

## SRU-Abfragen erklärt - Tutorial für Einsteiger\*innen (JupyterLite-Edition) 

Dieses Tutorial beschreibt, wie Sie mit Hilfe von JupyterLite und der Programmiersprache Python die SRU-Schnittstelle der DNB abfragen und mit den erhaltenen Antworten arbeiten können. Der Aufbau der Abfragen wird anhand von Beispielen erklärt und stützt sich auf die Dokumentation der SRU-Schnittstelle unter https://www.dnb.de/sru.

Das Tutorial ist wie folgt aufgebaut: 

* [1. Einrichten der Arbeitsumgebung](#Teil1) 
* [2. Abfragen verschiedener Datensätze der DNB](#Teil2)  
* [3. Aufbau einer gezielten Suche](#Teil3) 

<font color="red"><strong>Hinweis:</strong></font> JupyterLite ist eine schnelle, "leichtgewichtige" und sehr ressourcenschonende Coding-Umgebung. Dadurch stehen in JupyterLite nicht alle Funktionalitäten zur Verfügung und der Code des Tutorials wurde im Vegleich zur Jupyter Notebook-Version entsprechend angepasst. 

## 1. Einrichten der Arbeitsumgebung

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, werden zunächst die benötigten Python-Biblitoheken importiert: "urllib", "pyodide" und "js" für die Abfragen an die SRU-Schnittstelle in einer JupyterLite-Umgebung, "ElementTree" über "lxml" (als ET) und "BeautifulSoup", um die XML-Antworten der Schnittstelle besser verarbeiten zu können sowie "unicodedata" zur Verarbeitung der Zeichencodierung:

In [2]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree as ET

Die SRU-Schnittstelle der DNB ist unter der URL "https://services.dnb.de/sru" erreichbar. Ein Aufruf dieser Adresse im Browser zeigt den aktuellen Status sowie die Version der Schnittstelle an. 

## 2. Abfrage verschiedener Datensätze der DNB 

Die DNB bietet Ihre Daten auf drei verschiedene "Kataloge" gesplittet an, von denen immer einer für eine Abfrage ausgewählt werden muss. Dies geschieht über eine Erweiterung der o.g. Base-URL. Zur Verfügung stehen folgende Kataloge: 

* Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
* Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
* Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die Erweiterungen für die URL sind folgende: 

* DNB: https://services.dnb.de/sru/dnb
* DMA: https://services.dnb.de/sru/dnb.dma
* GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine Selbstbeschreibung im XML-Standardformat (https://services.dnb.de/sru/dnb?operation=explain&version=1.1) zurück. 

Mit Hilfe der Bibliothek "BeautifulSoup" kann die Antwort direkt in XML umgewandelt werden. 

<font color="red"><strong>Hinweis:</strong></font>  Um das Tutorial übersichtlich zu halten, wird die Ausgabe der Antwort im folgenden auf ersten 500 Zeichen gekürzt - die eigentliche Antwort ist länger und kann durch einfaches Löschen der Einschränkung "[0:500]" in der "print"-Zeile komplett angezeigt werden. Natürlich können auch andere Bereiche zur Anzeige gewählt werden. 

In [3]:
#URL der SRU-Schnittstelle der DNB: 
base_url = "https://services.dnb.de/sru/dnb"

#Anfrage - speichern der Antwort in die Variable "basic_request" und Überführung des Inhalts in die Variable "response":
basic_request =  await fetch(base_url)
content = await basic_request.text()

In [4]:
#Umwandeln in XML und Ausgabe der ersten 500 Zeichen: 
response = soup(content, features="xml")
print(response.prettify()[0:500])

<?xml version="1.0" encoding="utf-8"?>
<explainResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <record>
  <recordSchema>
   http://explain.z3950.org/dtd/2.0/
  </recordSchema>
  <recordPacking>
   xml
  </recordPacking>
  <recordData>
   <ns:explain id="Deutsche Nationalbibliothek" xmlns:ns="http://explain.z3950.org/dtd/2.0/">
    <ns:serverInfo protocol="sru" version="1.1">
     <ns:host>
      services.dnb.de
     </ns:host>
     <ns:port>
      443
     </ns:port>


Für eine Suchanfrage an die Daten der DNB wird nun über die Wahl der URL der Katalog definiert. Mit Hilfe der Variable *parameter* werden dann alle weiteren benötigten Parameter übergeben. 

Besonders relevant sind dabei die beiden Parameter 'query' : 'Klimawandel', sowie 'recordSchema' : 'MARC21-xml'. Statt "Klimawandel" kann hier jeder beliebige Suchbegriff eingetragen werden - auch Suchanfragen, die aus mehreren Wörtern bestehen, können hier mittels boolscher Operatoren übergeben werden. Die genaue Formulierung solcher kann unter https://www.dnb.de/sru nachgelesen werden. Statt "MARC21-xml" stehen außerdem noch "oai_dc" oder "RDFxml" als Ausgabeformate zur Verfügung (siehe weiter unten).   


In [5]:
base_url = "https://services.dnb.de/sru/dnb"
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'query': "Klimawandel"
         }
    
r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
r_text = await r.text()

response = soup(r_text, features="xml")
print(response.prettify()[0:1000])

<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  9959
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    MARC21-xml
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
     <leader>
      00000nam a22000008c 4500
     </leader>
     <controlfield tag="001">
      1147699615
     </controlfield>
     <controlfield tag="003">
      DE-101
     </controlfield>
     <controlfield tag="005">
      20221206110102.0
     </controlfield>
     <controlfield tag="007">
      tu
     </controlfield>
     <controlfield tag="008">
      171204s2027    gw ||||| |||| 00||||ger
     </controlfield>
     <datafield ind1=" " ind2=" " tag="015">
      <subfield code="a">
       17,N50
      </subfield>
      <subfield code="2">
       dnb
      </subfield>
     </datafield>
     <datafield 

Zu beachten ist, dass die Suche nach einem Stichwort über den 'query'-Befehl eine allgemeine Suche über alle Titeldaten darstellt. Die Suche ist allerdings nicht auf Titel oder ähnliches beschränkt, sondern durchsucht die Datensätze im Gesamtindex der Katalogdaten. Auch nach beispielsweise Autor*innennamen kann auf diese Art gesucht werden, jedoch muss bedacht werden, dass auch Titel, die den gesuchten Namen enthalten, in diesem Fall als Treffer ausgegeben werden.

## 3. Aufbau einer gezielten Suche 

Zur Eingrenzung der Suche auf bestimmte Angaben wie bspw. Titel oder Autor*in können unter anderem folgende Befehle genutzt werden:

tit= Suche im Titeleintrag
atr= Suche nach Verfasser*in (Person oder Organisation)
per= Suche nach Personen (in allen relevanten Feldern)
sw = Suche nach Schlagworten
jhr = Suche nach Erscheinungszeitraum
...
Eine detaillierte Übersicht über die verschiedenen Abfragemöglichkeiten gibt es unter https://www.dnb.de/expertensuche. Dabei können die unterschiedlichen Parameter auch beliebig in der Suchanfrage kombiniert werden - zu beachten ist hier lediglich, dass diese immer Teil der "query" sind.

Für die Ausgabe der Ergebnisse kann außerdem zwischen drei Formaten gewählt werden, indem der entsprechende Code hinter 'recordSchema' geändert wird:

MARC21-xml (XML-Variante von MARC 21)
oai_dc (DNB Casual - Auswahl von Dublin-Core-Elementen - nur für Titeldaten!)
RDFxml (RDF - Linked Data Service)
Eine Suchanfrage nach Titeln, die das Suchwort "Klimawandel" enthalten, im Jahr 2005 erschienen sind und im Format DNB Casual "oai_dc" angefragt wird, sieht wie folgt aus:

In [6]:
#Parameter, die der Anfrage übergeben werden: 
parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : 'tit=Klimawandel and jhr=2005',
             'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
r_text = await r.text()

#Parsen der Antwort "r" als XML in die neue Variable "response":
response = soup(r_text, features="xml")

#Schöne Ausgabe der ersten 1000 Zeichen: 
print(response.prettify()[0:1000])

<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  29
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    oai_dc
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
     <dc:title>
      Anpassung an den Klimawandel : Gründe, Folgen, Handlungsoptionen / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung ; Gtz, Deutsche Gesellschaft für Technische Zusammenarbeit (GTZ) GmbH
     </dc:title>
     <dc:creator>
      Deutschland / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung
     </dc:creator>
     <dc:publisher>
      Eschborn : GTZ
     </dc:publisher>
     <dc:date>
      2005
     </dc:date>
     <dc:iden

In Zeile 8 der Antwort befindet sich folgender XML-Block, der im Text zwischen den XML-Tags die Gesamtzahl der gefundenen Ergebnisse enthält:  
```
<numberOfRecords>
   ZAHL
</numberOfRecords>
```

Wenn man diese Information nicht im XML suchen möchte, kann auch der Code so angepasst werden, dass der entsprechende Abschnitt mithilfe des Zusatzes ".find('numberOfRecords')" gesucht wird. Der Zusatz ".find('suchtext')" sucht dabei nach dem ersten Element, welches den Suchtext in Klammern enthält. Für diesen Fall ist das ausreichend, da die Information nur einmal pro Anfrage von der SRU-Schnittstelle zurückgegeben wird. Wenn dagegen mehrere XML-Tags mit demselben Namen gesucht und ausgegeben werden sollen, muss ".find_all('suchtext')" genutzt werden. 

Im Anschluss kann der Inhalt zwischen den beiden "numberOfRecords"-Tags ausgegeben werdem, indem das Attribut ".text" an die Variable *number* angehängt wird. Zum Vergleich kann die gesamte Variable "number" (unten auskommentiert) ausgegeben werden:

In [7]:
number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')
#print(number)

29 Ergebnisse


Da die einzelzen Treffer bzw. Datensätze jeweils durch "record"-Tags gekennzeichnet sind, werden diese nun gesucht und in der Variable records zwischengespeichert. Zum Vergleich wird im Anschluss noch die Länge der Variable ausgeben, um überprüfen zu können, ob diese mit der Angabe unter "numberOfRecords" übereinstimmt.

<font color="orange"><strong>Wichtig!</strong></font>  Dies funkioniert zunächst nur bis zu einer Treffermenge von insgesamt 100! Auch bei größeren Treffermengen wird im folgenden maximal die Länge 100 angezeigt - wie man trotzdem größere Treffermengen sammeln kann, folgt etwas weiter unten in diesem Tutorial.

In [8]:
records = response.find_all('record')
print(len(records), 'Ergebnisse')

29 Ergebnisse


<font color="red"><strong>Hinweis:</strong></font> Wird statt DNB Casual (oai_dc) das Metadatenformat 'MARC21-xml' für die Ausgabe der Schnittstelle verwendet, sind die Datensätze im XML anders verschachtelt. In diesem Fall muss auch der Typ des Datensatzes bei jeder Suche nach 'record' um diese Angabe erweitert werden!
Bei einer Abfrage der Titeldaten über den Katalog /dnb muss ergänzt werden, dass es sich um bibliographische Datensätze handelt, bei einer Abfrage der GND enstprechend, dass es sich um Datensätze des Typs 'authority' handelt. Die Abfagen hierfür sehen wie folgt aus: 

MAR21-xml Titeldaten: <bold> `records = response.find_all('record', {'type':'Bibliographic'})` </bold> <br>
MAR21-xml Normdaten: <bold> `records = response.find_all('record', {'type':'Authority'})` </bold>


Die Ergebnisse werden als Liste gespeichert, d.h. die Variable records ist entsprechend eine Listenvariable. Die Ergebnisse stehen dabei jeweils an einem Listenplatz - bei 9 Ergebnissen gibt es in der Liste also 9 Einträge. Da der erste Eintrag allerdings an Listenplatz 0 steht, sind die Plätze von 0-8 mit den 9 Einträgen belegt. Dies ist wichtig, wenn einzelene Listenplätze adressiert werden sollen.

Der 3. Eintrag kann daher bspw. angezeigt werden, indem Listenplatz Nummer 2 aufgerufen wird:

In [9]:
print(records[2])

<record><recordSchema>oai_dc</recordSchema><recordPacking>xml</recordPacking><recordData><dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<dc:title>Die EU im Einsatz gegen den Klimawandel : der EU-Emissionshandel - ein offenes System, das weltweit Innovationen fördert / [Europäische Kommission]</dc:title>
<dc:creator>Europäische Kommission</dc:creator>
<dc:publisher>[Luxemburg] : [Amt für Amtliche Veröff. der Europ. Gemeinschaften]</dc:publisher>
<dc:date>2005</dc:date>
<dc:language>ger</dc:language>
<dc:identifier xmlns:tel="http://krait.kb.nl/coop/tel/handbook/telterms.html" xsi:type="tel:ISBN">92-894-9187-6 geh.</dc:identifier>
<dc:identifier xsi:type="dnb:IDN">992017882</dc:identifier>
<dc:subject>360 Soziale Probleme, Sozialdienste, Versicherungen</dc:subject>
<dc:subject>330 Wirtschaft</dc:subject>
<dc:format>20 S.</dc:format>
</d

Um das Ganze einfacher zu nutzen, können alle Schritte in einer Funktion zusammengeführt werden, die dann nur noch die jeweilige Abfrage erhalten muss:

In [39]:
#Funktion
async def dnb_sru_short(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    parameter = {'recordSchema' : 'oai_dc',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
    content = await r.text()
    xml = soup(content, features="xml")
    records = xml.find_all('record')
    
    return records
    


In [41]:
#Formulierung der Abfrage: 
myquery = await dnb_sru_short('tit=Klimawandel and jhr=2005') #Aufruf der Funktion 'sru-dnb' mit der Abfrage 'tit=Klimawandel...'
print(len(myquery), "Ergebnisse")

29 Ergebnisse


Da im Vorfeld die Menge der Ergebnisse nicht bekannt ist, aber häufig Ergebnismengen von über 100 Treffern vorkommen, kann die Funktion noch etwas angepasst werden. Dazu wird mit Hilfe der Information im Tag "numberOfRecords" zunächst abgefragt, wie viele Treffer insgesamt gefunden wurden. Werden bis zu 100 Treffer gemeldet, gibt es keinen Änderungsbedarf und die Funktion kann durchlaufen und die gefundenen Treffer zurückgeben. Werden jedoch mehr als 100 Treffer gefunden, wird die Anfrage in 100er Schritte aufgeteilt und die Ergebnisse jeweils zwischengespeichert. Dies geschieht mithilfe einer Schleife, die nach jeder neuen Abfrage überprüft, ob noch weitere Treffer "geholt" werden müssen und diese in 100er Schritten "einsammelt". Sobald alle Teile der Anfrage verarbeitet wurden, wird das gesammelte Ergebnis übergeben. Die Funktion sieht dann folgendermaßen aus:

In [44]:
#Funktion
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    parameter = {'recordSchema' : 'oai_dc',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
    content = await r.text()
    xml = soup(content, features="xml")
    number = int(xml.find('numberOfRecords').text)
    records = xml.find_all('record')
    
    if number <= 101:       # wurden maximal 100 Treffer gefunden? Wenn ja, erfolgt direkt die Rückgabe.
        return records
    
    else:                   # wurden mehr als 100 Treffer gefunden, wird hier die Schleife gestartet.
        num_results = 100
        i = 101
        
        while num_results == 100:
            
            parameter.update({'startRecord': i}) 
            r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
            content = await r.text()
            xml = soup(content, features="xml")
            new_records = xml.find_all('record')
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records
    

Diese Funktion wird auch bereits im Tutorial "Wie können Daten mittels der SRU-Schnittstelle abgerufen werden?" verwendet.

Eine Abfrage für das Titelstichwort "Klimawandel" kombiniert mit dem Jahr 2019 ergibt folgende Treffermenge:

In [45]:
myquery = await dnb_sru('tit=Klimawandel and jhr=2019')
print(len(myquery), "Ergebnisse")

305 Ergebnisse


Die so abgerufenen Treffer können entweder direkt weiterverarbeitet oder lokal zwischengespeichert werden. Der folgende Codeblock erstellt eine entsprechende Datei und legt den Inhalt der Abfrage dort als XML ab: 

In [46]:
with open('sru_abfrage_klimawandel.xml', 'w', encoding="utf-8") as f:
    f.write(str(myquery))

<font color="red"><strong>Hinweis:</strong></font> Wie oben bereits erwähnt, entnimmt die Schleife jeweils die bibliografischen Datensätze aus den XML-Antworten der Schnittstelle und legt diese in eine Liste ab. Es handelt sich daher bei der Variable myquery nicht um valides XML (das alle Formatregeln berücksichtigt), da ein Root-Element fehlt. Für die weitere Verarbeitung der abgerufenen Datensätze zu Analysezwecken ist dies jedoch unerheblich.